In [1]:
import peft
import torch
import transformers
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType

/home/shna94/naver_databox_test_bed/llama/alpaca/stanford_alpaca/alpaca_env/lib64/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model(model, pre_trained_state_path):
    load_state = torch.load(pre_trained_state_path)
    model.load_state_dict(load_state)
    
    return model, load_state


def check_param(model, load_state):
    for name, param in model.named_parameters():
        if not param.equal(load_state[name.replace('.default', '')]):
            print(name + 'is not equal')


# DEVICE = "cuda"
DEVICE = "cpu"

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "kakaobrain/kogpt",
        revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
        bos_token="[BOS]",
        eos_token="[EOS]",
        unk_token="[UNK]",
        pad_token="[PAD]",
        mask_token="[MASK]",
        model_max_length=512
#         cache_dir=training_args.cache_dir,
    )

In [7]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    '/ailab/share/kogpt/kogpt-ft-3',
    revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
    pad_token_id=tokenizer.eos_token_id,
    # torch_dtype="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(DEVICE)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.46s/it]


In [8]:
import os
import torch
import pandas as pd
import numpy as np
import re
import random
from random import sample
from tqdm import tqdm

In [9]:
# SEED_NUM = None
SEED_NUM = 1234
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

In [11]:
# 데이터 전처리 준비
DATA_IN_PATH = '/home/shna94/naver_databox_test_bed/llama/alpaca/KoAlpaca/data_in/KOR'
DATA_OUT_PATH = '/home/shna94/naver_databox_test_bed/llama/alpaca/KoAlpaca/data_out/KOR'

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_train.txt')
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_test.txt')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [21]:
print('데이터 positive 라벨: ', '긍정')
print('데이터 negative 라벨: ', '부정')
print('\n학습 예시 케이스 구조: ', '문장: 오늘 기분이 좋아\n감정: 긍정\n')
print('\n모델 최대 토큰 길이: ', model.config.max_position_embeddings)

데이터 positive 라벨:  긍정
데이터 negative 라벨:  부정

학습 예시 케이스 구조:  문장: 오늘 기분이 좋아
감정: 긍정


모델 최대 토큰 길이:  2048


In [22]:
sent_lens = [len(tokenizer(s).input_ids) for s in tqdm(train_data['document'])]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 149995/149995 [00:18<00:00, 8321.02it/s]


Few shot 케이스 토큰 평균 길이:  18.40927364245475
Few shot 케이스 토큰 최대 길이:  146
Few shot 케이스 토큰 길이 표준편차:  15.227606399857397
Few shot 케이스 토큰 길이 80 퍼센타일:  25.0


In [23]:
train_fewshot_data = []

for train_sent, train_label in tqdm(train_data[['document', 'label']].values):
    tokens = tokenizer(train_sent).input_ids

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 149995/149995 [00:17<00:00, 8638.53it/s]


In [25]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [40]:
# Full Dataset
# sample_size = len(test_data)

# Sampled Dataset
# sample_size = 500
sample_size = 10

train_fewshot_samples = []

for _ in range(sample_size):
    fewshot_examples = sample(train_fewshot_data, 10)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)

In [54]:
def build_prompt_text(sent):
    return "\n\n문장: " + sent + '\n감정:'

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

prompt_text = ''
for example_text, example_label in train_fewshot_samples[i]:
    cleaned_example_text = clean_text(example_text)
    appended_prompt_example_text = build_prompt_text(cleaned_example_text)
    appended_prompt_example_text += ' 긍정' if example_label == 1 else ' 부정'
    prompt_text += appended_prompt_example_text
    
print(prompt_text)    



문장: 당시엔 왜그렇게 웃겼는지ㅋㅋㅋㅋㅋ 어쨌든 점ㅋㅋㅋㅋ
감정: 긍정
문장: 대박이요김정은언니정준호 오빠김흥수오빠너무 너무 좋아요
감정: 긍정
문장: 한번만 봐준다는 의미해서 점준다
감정: 부정
문장: 조카랑 보기 좋았던따뜻한 애니였어여
감정: 긍정
문장: 이게 년도 영화라니충격 이 영화 보고나니 금욕주의자가 될것 같다
감정: 긍정
문장: 진짜 개 노잼 뭘 말하는지 모르겠다
감정: 부정
문장: 야하기만한 영화는 아닌듯
감정: 긍정
문장: 해학을 알면보인다
감정: 긍정
문장: 점을 주고싶은 마음
감정: 부정
문장: 원작의 긴장감을 제대로 살려내지못했다
감정: 부정


In [41]:

real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정' if example_label == 1 else ' 부정' + '\n'
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt").to(DEVICE)
    token_ids, attn_mask = tokens.input_ids, tokens.attention_mask
    gen_tokens = model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()

    pred_tokens.append(pred)
    real_labels.append('긍정' if test_label == 1 else '부정')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:00<00:00,  6.07s/it]


In [30]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.71
